# Simulating the Luria-Delbrück Experiment

### Are mutations induced or do they arise spontaneously?

This was the question faced by Salvador Lauria and Max Delbrück in 1943 when they devised their seminal fluctuation test experiment. As a modern student of biology, the answer may seem obvious, but at the time it remained a mystery. The inspiration for such a question comes from observing the interaction between bacteria and bacteria phages (i.e. viruses that infect bacteria). A colony of bacteria growing in a flask will eventually turn the medium in the flask cloudy as the number of cells increases. It was noted that adding an equal number of bacteria phages to the flask would clear the water, as the bacteria phages lysed open the bacterial cells. However, after some time the medium in the flask would become cloudy again. This is because the bacterial colony had developed mutations in its genomes that granted resistance to the phages. The question was then "how did these resistance mutations come about?"

<img src='graphics/Grupo_de_los_Fagos-239509034.jpeg'/> 



Lauria and Delbrück proposed two simple models for the evolution of resistance:

**1) Mutations in the bacteria genomes resulting in resistance occurred in response to the bacteria phage (i.e. mutations were induced)**

or 

**2) Mutations occurred spontaneously during colony growth, and bacteria carrying these mutations were already resistant prior to exposure to the phage**

***

To determine which model was correct, they devised an experiment known as the **fluctuation test**. This test proceeded as follows:

First a colony of non-resistant bacteria were grown in a flask. These bacteria were then used to innoculate media in (e.g. 11) identical test tubes. The bacteria where then allowed to grow in these test tubes until they were at a certain cell density. The test tubes were then split into two groups - for the purposes of this notebook, we will call these groups A and B. One tube was put into group A, and from this tube 10 plates, already containing bacteria phages, were innoculated with bacteria. The other 10 tubes constituted group B, and from each tube a single plate, again contaning bacteria phages, was inoculated with bacteria. The number of resistant colonies were then counted on each plate.

<img src='graphics/luria1.webp'/>

**If the first model is true, then we would expect that the variance in the number of resistant colonies per plate would be the same for parts A and B.** This is because resistance is induced at a certain rate once the bacteria are plated out, and would not depend on what had happened while the bacteria grew in the test tubes. In other words, each plate would be an independant sample, regardless of if the bacteria came from the same test tube or not - hence equal variances.

**If the second model is true, then we would expect the variance to be far greater for part B than for part A.** This is due to the fact that the resistance mutation occurs spontaneously during growth in the test tube. When plating out part A, the number of resistant colonies is representative of the frequency of resistance in the single tube, so each plate is not independant. On the other hand, in part B each tube has had an independant phase of growth, during which resistance may have spontaneously evolved early on, resulting in a high frequency of resistant colonies, or later on, resulting in few resistant colonies. This independance results in a large variance.  

<img src='graphics/The-Luria-and-Delbrueck-experiment-In-1943-Luria-and-Delbrueck-devised-an-experiment-to-4102026779.png'/>

When Luria and Delbück performed this experiment, they found the later. However, what if you were in their place? How would you decide on the specifics of this experiment? How many replicates would you need, or how many bacteria would you plate out? How many generations would need to be run to see an effect at the mutation rate you would expect? Additionally, the experiment relies on the assumption that there is no standing genetic variance for resistance in the ancestral bacteria. How would violating this assumption effect the results?

To answer these questions and gain an intuition for the limitations of this experimental design, **lets simulate!!**

***

#### But how can we go about simulating something like this? 
Heres a recommendation for how to start with a simulation in general:

1) **Develop a good mental model** - make sure you understand the biology and the techical aspects of what you want to simulate.
2) **Break it down into subprocesses** - what are the different steps involved going from initialization to final product?
3) **Describe in (painful) detail how each subprocess works** - pretend like you to have to describe it to an alien who has never been to earth and knows nothing how the subprocess works. No detail is too insignificant! Be sure to think about potential biases.
4) **Convert those descriptions into pseudocode, keeping in mind which data sctructures you will use** - think about which functions you will use where, and how you will store and manage your data. What is an efficent way to code this process? Do this with a pen and paper to figure out exactly how your algorithm will work.
5) **Make the pseudocode actual code** 

***

#### Lets see how we can do this for the Luria-Delbrück experiment 
For now we will just focus on the 2nd model, the one of spontaneous mutation.

The subprocesses and their descriptions are:
- **Initializing an ancestral population**
    - Create a flask with a large population of non-resistant bacteria
- **Sampling the ancestral population to create the test tubes**
    - Randomly select n_0 cells from the ancestral flask and put these cells into new medium. Create 1 tube for part A and r tubes for part B. 
- **Growing the cultures**
    - In each generation, every cell produces two children cells. When the children cells are formed, there is a chance that each child will mutate some resistance. These children then become the parents for the next generation. This process repeats for T generations.  
- **Plating out**
    - For part A:
        - Randomly sample n_sample cells without replacement for the A tube for each plate, for r number of plates
    - For part B:
        - Randomly sample n_sample cells without replacement from each of r tubes for part B to create one plate per tube
- **Computing the variance**
    - For each plate, count the number of resistant colonies, then calculate the variances for parts A and B


#### Now lets see how we would implement this in Python


In [ ]:
def run_generation(parents, mu):
    children_1 = parents + np.random.binomial(1,mu, size = parents.shape)
    children_2 = parents + np.random.binomial(1,mu, size = parents.shape)
    return np.concatenate((children_1,children_2))

In [ ]:
def sample_no_replacement(arr,n):
    np.random.shuffle(arr)
    sample = arr[:n]
    rest = arr[n:]
    return sample, rest

In [ ]:
mu = 1e-4
T = 15
r = 50
n_0 = 100
n_sample = 10_000

anc_pop = np.zeros(1000000)

#### A side ####
parents = np.random.choice(anc_pop, size = 100, replace = False)
# run generations
for gen in range(T):
    parents = run_generation(parents,mu)
# Bianarize parents
parents = (parents > 0).astype(int)
# sample 50 times from tube
plates = []
for i in range(r):
    sample, parents = sample_no_replacement(parents,10000)
    plates.append(sample.sum())
#variance of the A tube
var_a = np.var(np.array(plates))

#### B side ####
plates = []
for i in range(r): # simulating each tube
    # sample parents
    parents = np.random.choice(anc_pop, size = 100, replace = False)
    # reproduce
    for gen in range(T):
        parents = run_generation(parents,mu)
    # Bianarize parents 
    parents = (parents > 0).astype(int)
    # sample from tube
    plate = np.random.choice(parents, size = 10000, replace = False)
    plates.append(plate.sum())
# variance of the beta tubes
var_b = np.var(np.array(plates))
    
print(var_a,var_b)

In [ ]:
def sim_tube(mu,T,n_0):
    n_wt = n_0
    n_res = 0
    for gen in range(T):
        n_res = 2*n_res + np.random.binomial(2*n_wt, mu)
        n_wt = n_0*2**gen - n_res
    return np.array([n_res, n_wt])

mu = 1e-6
T = 15
r = 50
n_0 = 100
n_sample = 10_000

# part A
tube = sim_tube(mu,T,n_0)
res_A = np.empty(r,int)
for i in range(r):
    res_A[i] = np.random.hypergeometric(tube[0],tube[1],n_sample)
    tube[0] = tube[0] - res_A[i]
    tube[1] = tube[1] - (n_sample - res_A[i])


# part B
res_B = np.empty(r,int)
for i in range(r):
    tube = sim_tube(mu,T,n_0)
    res_B[i] = np.random.hypergeometric(tube[0],tube[1],n_sample)
    
print(np.var(res_A), np.var(res_B))